In [1]:
import numpy as np
import pandas as pd 

In [2]:
df1=pd.read_csv("dataset-1.csv") 
df2=pd.read_csv("dataset-2.csv") 
df3=pd.read_csv("dataset-3.csv")

In [3]:
df1.head()

,id_1,id_2,route,moto,car,rv,bus,truck
0,829,827,1,2.05,4.14,4.14,10.1,15.2
1,829,821,4,6.63,13.26,13.26,32.4,48.5
2,829,804,7,14.41,28.92,28.92,64.7,97.0
3,829,822,6,5.90,11.81,11.81,28.8,43.2
4,829,826,9,2.87,5.81,5.81,14.2,21.2


In [4]:
df1.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id_1    341 non-null    int64  
 1   id_2    341 non-null    int64  
 2   route   341 non-null    int64  
 3   moto    341 non-null    float64
 4   car     341 non-null    float64
 5   rv      341 non-null    float64
 6   bus     341 non-null    float64
 7   truck   341 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 21.4 KB


In [5]:
df2.head() 

,id,name,id_2,startDay,startTime,endDay,endTime,able2Hov2,able2Hov3,able3Hov2,able3Hov3,able5Hov2,able5Hov3,able4Hov2,able4Hov3
0,1040000,Montgomery,-1,Monday,05:00:00,Wednesday,10:00:00,3.0,3.0,-1.0,-1,3,3,3,3
1,1040010,Black,-1,Monday,10:00:00,Friday,15:00:00,6.0,6.0,-1.0,-1,6,6,6,6
2,1040020,Emerald,-1,Thursday,15:00:00,Friday,19:00:00,3.0,3.0,-1.0,-1,3,3,3,3
3,1040030,Foley,-1,Monday,19:00:00,Friday,23:59:59,6.0,6.0,-1.0,-1,6,6,6,6
4,1050000,Whittier,1050001,Saturday,00:00:00,Sunday,23:59:59,6.0,6.0,NaN,-1,6,6,6,6


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39514 entries, 0 to 39513
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         39514 non-null  int64  
 1   name       39514 non-null  object 
 2   id_2       39514 non-null  int64  
 3   startDay   39514 non-null  object 
 4   startTime  39514 non-null  object 
 5   endDay     39514 non-null  object 
 6   endTime    39514 non-null  object 
 7   able2Hov2  37708 non-null  float64
 8   able2Hov3  37709 non-null  float64
 9   able3Hov2  37709 non-null  float64
 10  able3Hov3  39514 non-null  int64  
 11  able5Hov2  39514 non-null  int64  
 12  able5Hov3  39514 non-null  int64  
 13  able4Hov2  39514 non-null  int64  
 14  able4Hov3  39514 non-null  int64  
dtypes: float64(3), int64(7), object(5)
memory usage: 4.5+ MB


In [8]:
df3.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


In [9]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_start  44 non-null     int64  
 1   id_end    44 non-null     int64  
 2   distance  44 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.2 KB


## Question 1: Distance Matrix Calculation
- Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

- The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [10]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(df):
    # Get unique toll locations
    unique_locations = sorted(set(df['id_start']) | set(df['id_end']))
    num_locations = len(unique_locations)

## Create an empty distance matrix
    distance_matrix = pd.DataFrame(np.zeros((num_locations, num_locations)), index=unique_locations, columns=unique_locations)

## Iterate through the dataset to calculate cumulative distances
    for index, row in df.iterrows():
        location_A, location_B, distance = row['id_start'], row['id_end'], row['distance']

## Update distances bidirectionally
        distance_matrix.loc[location_A, location_B] += distance
        distance_matrix.loc[location_B, location_A] += distance

## Calculate cumulative distances for indirect routes
    for i in range(num_locations):
        for j in range(num_locations):
            for k in range(num_locations):
                if distance_matrix.iloc[j, i] > 0 and distance_matrix.iloc[i, k] > 0:
                    if distance_matrix.iloc[j, k] == 0 or distance_matrix.iloc[j, k] > distance_matrix.iloc[j, i] + distance_matrix.iloc[i, k]:
                        distance_matrix.iloc[j, k] = distance_matrix.iloc[j, i] + distance_matrix.iloc[i, k]

## Set diagonal values to 0
    np.fill_diagonal(distance_matrix.values, 0)

    return distance_matrix

# Example usage with df1
result_filtered_routes = calculate_distance_matrix(df3)
result_filtered_routes 

,1001400,1001402,1001404,1001406,1001408,1001410,1001412,1001414,1001416,1001418,...,1001462,1001464,1001466,1001468,1001470,1001472,1001488,1004354,1004355,1004356
1001400,0.0,9.7,29.9,45.9,67.6,78.7,94.3,112.5,125.7,139.3,...,371.8,398.5,407.0,417.7,428.3,444.3,264.5,270.5,272.5,268.5
1001402,9.7,0.0,20.2,36.2,57.9,69.0,84.6,102.8,116.0,129.6,...,362.1,388.8,397.3,408.0,418.6,434.6,254.8,260.8,262.8,258.8
1001404,29.9,20.2,0.0,16.0,37.7,48.8,64.4,82.6,95.8,109.4,...,341.9,368.6,377.1,387.8,398.4,414.4,234.6,240.6,242.6,238.6
1001406,45.9,36.2,16.0,0.0,21.7,32.8,48.4,66.6,79.8,93.4,...,325.9,352.6,361.1,371.8,382.4,398.4,218.6,224.6,226.6,222.6
1001408,67.6,57.9,37.7,21.7,0.0,11.1,26.7,44.9,58.1,71.7,...,304.2,330.9,339.4,350.1,360.7,376.7,196.9,202.9,204.9,200.9
1001410,78.7,69.0,48.8,32.8,11.1,0.0,15.6,33.8,47.0,60.6,...,293.1,319.8,328.3,339.0,349.6,365.6,185.8,191.8,193.8,189.8
1001412,94.3,84.6,64.4,48.4,26.7,15.6,0.0,18.2,31.4,45.0,...,277.5,304.2,312.7,323.4,334.0,350.0,170.2,176.2,178.2,174.2
1001414,112.5,102.8,82.6,66.6,44.9,33.8,18.2,0.0,13.2,26.8,...,259.3,286.0,294.5,305.2,315.8,331.8,152.0,158.0,160.0,156.0
1001416,125.7,116.0,95.8,79.8,58.1,47.0,31.4,13.2,0.0,13.6,...,246.1,272.8,281.3,292.0,302.6,318.6,138.8,144.8,146.8,142.8
1001418,139.3,129.6,109.4,93.4,71.7,60.6,45.0,26.8,13.6,0.0,...,232.5,259.2,267.7,278.4,289.0,305.0,125.2,131.2,133.2,129.2


## Question 2: Unroll Distance Matrix¶
- Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

- All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [11]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Create an empty DataFrame to store unrolled distance values
    unrolled_distance_df = pd.DataFrame(columns=['id_start', 'id_end', 'distance'])

    # Iterate through the distance matrix and append values to the new DataFrame
    for start_id in distance_matrix.index:
        for end_id in distance_matrix.columns:
            if start_id != end_id:
                new_row = pd.DataFrame({
                    'id_start': [start_id],
                    'id_end': [end_id],
                    'distance': [distance_matrix.at[start_id, end_id]]
                })
                unrolled_distance_df = pd.concat([unrolled_distance_df, new_row], ignore_index=True)

    return unrolled_distance_df

# Example usage with the distance matrix obtained from the previous question
result_unrolled_distance = unroll_distance_matrix(result_filtered_routes) 
result_unrolled_distance 

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001400,1001404,29.9
2,1001400,1001406,45.9
3,1001400,1001408,67.6
4,1001400,1001410,78.7
...,...,...,...
1801,1004356,1001470,159.8
1802,1004356,1001472,175.8
1803,1004356,1001488,4.0
1804,1004356,1004354,2.0


## Question 3: Finding IDs within Percentage Threshold
- Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

- Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [12]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(distance_matrix, reference_value):
   
    # Calculate the average distance for the reference value
    avg_distance = distance_matrix[distance_matrix['id_start'] == reference_value]['distance'].mean()

    # Calculate the threshold values
    lower_threshold = avg_distance - (avg_distance * 0.1)
    upper_threshold = avg_distance + (avg_distance * 0.1)

    # Filter the values within the 10% threshold 
    within_threshold = distance_matrix[(distance_matrix['distance'] >= lower_threshold) & (distance_matrix['distance'] <= upper_threshold)]["id_start"].unique()

    return sorted(within_threshold)     

# Example usage with the distance matrix obtained from Question 2
reference_value = 1001400  # You can change this to any valid reference value
result_ids_within_threshold = find_ids_within_ten_percentage_threshold(result_unrolled_distance, reference_value)
result_ids_within_threshold

[1001400,
 1001402,
 1001404,
 1001406,
 1001408,
 1001410,
 1001412,
 1001414,
 1001416,
 1001418,
 1001420,
 1001422,
 1001424,
 1001426,
 1001428,
 1001430,
 1001432,
 1001434,
 1001436,
 1001437,
 1001438,
 1001440,
 1001442,
 1001444,
 1001446,
 1001448,
 1001450,
 1001452,
 1001454,
 1001456,
 1001458,
 1001460,
 1001461,
 1001462,
 1001464,
 1001466,
 1001468,
 1001470,
 1001472,
 1001488,
 1004354,
 1004355,
 1004356]

## Question 4: Calculate Toll Rate
- Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

- The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

- 0.8 for moto
- 1.2 for car
- 1.5 for rv
- 2.2 for bus
- 3.6 for truck

In [13]:
import pandas as pd

def calculate_toll_rate(distance_matrix):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    # Create a copy of the input DataFrame to avoid modifying the original
    toll_rate_matrix = distance_matrix.copy()

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        toll_rate_matrix[vehicle_type] = distance_matrix['distance'] * rate_coefficient

    return toll_rate_matrix[['id_start', 'id_end', 'moto', 'car', 'rv', 'bus', 'truck']]

# Example usage with the distance matrix obtained from Question 1
result_toll_rate_matrix = calculate_toll_rate(result_unrolled_distance) 
result_toll_rate_matrix

,id_start,id_end,moto,car,rv,bus,truck
0,1001400,1001402,7.76,11.64,14.55,21.34,34.92
1,1001400,1001404,23.92,35.88,44.85,65.78,107.64
2,1001400,1001406,36.72,55.08,68.85,100.98,165.24
3,1001400,1001408,54.08,81.12,101.40,148.72,243.36
4,1001400,1001410,62.96,94.44,118.05,173.14,283.32
...,...,...,...,...,...,...,...
1801,1004356,1001470,127.84,191.76,239.70,351.56,575.28
1802,1004356,1001472,140.64,210.96,263.70,386.76,632.88
1803,1004356,1001488,3.20,4.80,6.00,8.80,14.40
1804,1004356,1004354,1.60,2.40,3.00,4.40,7.20


## Question 5: Calculate Time-Based Toll Rates
- Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

- The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

- start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
- start_time and end_time must be of type datetime.time() with the values from time range given below.
- Modify the values of vehicle columns according to the following time ranges:

- Weekdays (Monday - Friday):

- From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
- From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
- From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
- Weekends (Saturday and Sunday):

- Apply a constant discount factor of 0.7 for all times.
- For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [14]:
import pandas as pd 
def calculate_time_based_toll_rates(df):
    # Function to calculate toll rates based on time intervals
    def apply_time_based_toll_rates(row):
        # Define time intervals and corresponding factors
        def get_time_interval(hour):
            if hour < 10:
                return '00:00:00', '10:00:00', 0.8
            elif 10 <= hour < 18:
                return '10:00:00', '18:00:00', 1.2
            else:
                return '18:00:00', '23:59:59', 0.8

        # Apply toll rates based on time intervals for each row
        for index, row_data in row.iterrows():
            start_day = int(str(row_data['id_start']).split('.')[0][-2:]) if not pd.isnull(row_data['id_start']) else None
            end_day = int(str(row_data['id_end']).split('.')[0][-2:]) if not pd.isnull(row_data['id_end']) else None

            if start_day is not None and end_day is not None:
                for hour in range(24):
                    start_time, end_time, factor = get_time_interval(hour)

                    # Apply discount factor to vehicle columns based on weekdays/weekends and time intervals
                    mask = (
                        (row_data['start_day'] == start_day) &
                        (pd.to_datetime(row_data['start_time']) <= pd.to_datetime(end_time)) &
                        (pd.to_datetime(row_data['end_time']) >= pd.to_datetime(start_time))
                    )

                    # Apply discount factor to vehicle columns based on weekdays/weekends and time intervals
                    if start_day < 5:  # Weekdays
                        row.loc[index, ['moto', 'car', 'rv', 'bus', 'truck']] *= factor
                    else:  # Weekends
                        row.loc[index, ['moto', 'car', 'rv', 'bus', 'truck']] *= 0.7

        return row

    # Extracting day and time components from 'id_start' and 'id_end' columns
    df[['start_day', 'start_time']] = df['id_start'].astype(str).str.extract(r'(\d{6})(\d{2})').astype(float)
    df[['end_day', 'end_time']] = df['id_end'].astype(str).str.extract(r'(\d{6})(\d{2})').astype(float)

    # Apply the toll rate function to the DataFrame
    result_df = apply_time_based_toll_rates(df.copy())

    return result_df

# Assuming 'calculated_toll_rates' is your DataFrame with columns 'id_start', 'id_end', 'distance', 'moto', 'car', 'rv', 'bus', 'truck'
# Call the function with your DataFrame
result_df = calculate_time_based_toll_rates(result_toll_rate_matrix) 
result_df  


,id_start,id_end,moto,car,rv,bus,truck,start_day,start_time,end_day,end_time
0,1001400,1001402,0.939186,1.408779,1.760973,2.582761,4.226336,NaN,NaN,NaN,NaN
1,1001400,1001404,2.895016,4.342524,5.428155,7.961293,13.027571,NaN,NaN,NaN,NaN
2,1001400,1001406,4.444188,6.666282,8.332853,12.221517,19.998846,NaN,NaN,NaN,NaN
3,1001400,1001408,6.545253,9.817880,12.272350,17.999446,29.453639,NaN,NaN,NaN,NaN
4,1001400,1001410,7.619991,11.429987,14.287484,20.954976,34.289961,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1801,1004356,1001470,0.024492,0.036738,0.045922,0.067352,0.110213,NaN,NaN,NaN,NaN
1802,1004356,1001472,0.026944,0.040416,0.050520,0.074096,0.121248,NaN,NaN,NaN,NaN
1803,1004356,1001488,0.000613,0.000920,0.001149,0.001686,0.002759,NaN,NaN,NaN,NaN
1804,1004356,1004354,0.000307,0.000460,0.000575,0.000843,0.001379,NaN,NaN,NaN,NaN
